# Import code from gitHub

In [0]:
import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None
_git_branch = 'document-parser-lib'


def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')


#----
import matplotlib as mpl
from documents import TextMap
from renderer import HtmlRenderer
from legal_docs import DocumentJson
 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range)

   
renderer_ = DemoRenderer()

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag.span
    _map = cd.tokenization_maps[tag.span_map]
    print(tag)
 
  

# Find contract Subject 

### Load pre-embedded test doc

In [0]:
import pickle
import unittest
import warnings

from contract_parser import ContractAnlysingContext, ContractDocument
from contract_patterns import ContractPatternFactory
from documents import TextMap
from legal_docs import LegalDocument
from ml_tools import SemanticTag, filter_values_by_key_prefix

import nltk
nltk.download('punkt')
 
#LOAD DOC 
with open('/content/nlp_tools/tests/2. Договор по благ-ти Радуга.docx.pickle', 'rb') as handle:
  doc = pickle.load(handle)

with open('/content/nlp_tools/tests/contract_pattern_factory.pickle', 'rb') as handle:
  factory = pickle.load(handle)


ctx = ContractAnlysingContext(embedder={}, renderer=None, pattern_factory=factory)
ctx.verbosity_level = 3

#Find sections
ctx.sections_finder.find_sections(doc, 
                                  ctx.pattern_factory, 
                                  ctx.pattern_factory.headlines,
                                  headline_patterns_prefix='headline.')

#


In [0]:
print (doc.paragraphs[1].header.value)

### make_subject_attention_vector_3
найдена секция "предмет договора" (subj)

In [0]:
from ml_tools import SemanticTag, FixedVector, filter_values_by_key_prefix, rectifyed_sum
from structures import ContractSubject

section = doc.sections['subj'].body
 
section.calculate_distances_per_pattern(ctx.pattern_factory, merge=True, pattern_prefix='x_ContractSubject')

all_subjects_vectors = filter_values_by_key_prefix(section.distances_per_pattern_dict, 'x_ContractSubject')
all_subjects_mean: FixedVector = rectifyed_sum(all_subjects_vectors)

subject_kind =  ContractSubject.Charity
subject_attention_vector: FixedVector = ctx.make_subject_attention_vector_3(section, subject_kind,
                                                                                   all_subjects_mean)

renderer_.render_color_text(section.tokens, subject_attention_vector)

### make_subject_attention_vector_3
первые 1500 слов, когда секция "предмет договора" не найдена

In [0]:
from ml_tools import SemanticTag, FixedVector, filter_values_by_key_prefix, rectifyed_sum
from structures import ContractSubject

#Finding subject in first 1500 words
section = doc.subdoc_slice(slice(0, 1500))

section.calculate_distances_per_pattern(ctx.pattern_factory, merge=True, pattern_prefix='x_ContractSubject')

all_subjects_vectors = filter_values_by_key_prefix(section.distances_per_pattern_dict, 'x_ContractSubject')
all_subjects_mean: FixedVector = rectifyed_sum(all_subjects_vectors)
renderer_.render_color_text(section.tokens, all_subjects_mean)

In [0]:

subject_kind =  ContractSubject.Charity
subject_attention_vector: FixedVector = ctx.make_subject_attention_vector_3(section, subject_kind,
                                                                                   all_subjects_mean)


renderer_.render_color_text(section.tokens, subject_attention_vector)


## Find subject sentence (span, region, paragraph)

In [0]:


#ACHTUNG: this code moved to gitHub codebase, see _find_most_relevant_paragraph

attention_vector = subject_attention_vector
paragraph_attention_vector = np.zeros_like (attention_vector)
top_index=0
for i in np.nonzero(attention_vector)[0]:
  par = section.tokens_map.sentence_at_index(i)
  paragraph_len = par[1]-par[0]
  if paragraph_len:
    # TODO: Next line is weird
    # Calculate density of the matches per paragraph:
    density = attention_vector[i] / paragraph_len
    paragraph_attention_vector[par[0]: par[1]] += attention_vector[i] + density

    if paragraph_attention_vector[par[0]] > paragraph_attention_vector[top_index]:
      top_index = par[0]

par = section.tokens_map.sentence_at_index(top_index)
renderer_.render_color_text(section.tokens_cc, paragraph_attention_vector)


In [0]:
doc.tokens_map.text_range(par)

# export JSON

In [0]:
from legal_docs import DocumentJson

ctx.analyze_contract_doc(doc)

_path = '____.json'
with open(_path, 'w') as file:
  jjj = DocumentJson(doc)
  _j= json.dumps(jjj.__dict__, indent=4, ensure_ascii=False, default=lambda o: '<not serializable>')
  file.write(_j)
  print(f'saved file to {_path}')

### Render single test Json

In [0]:
def print_semantic_tag(tag: SemanticTag, map: TextMap):
  print('-->', f'{tag.kind} \t\t [{tag.value}]\t [{map.text_range(tag.span)}]')


def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
 
  for tag in cd.tags:
    if tag.kind !='headline':
     print_semantic_tag(tag, wordsmap)
 

In [0]:


def json2html(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['words'])
  markup_vector = np.zeros(len(wordsmap))
 
  for tag in cd.attributes:
    if 'span' in  cd.attributes[tag]:
      span = cd.attributes[tag]['span']
      markup_vector[span[0]:span[1]] += 0.5
    else:
      warnings.warn(f'{tag} has no span')

  return renderer_.to_color_text(wordsmap.tokens, markup_vector, _range=(0, 1))
  
 


def load_and_show_json(filename):
  with open(filename, 'r') as json_file:
    data_str = json_file.read()
    # print(str(data_str))
    json_obj:DocumentJson = DocumentJson.from_json( data_str)
    html = json2html(json_obj)
    display(HTML(html))

load_and_show_json(_path)


## Export all contracts to JSON

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
from embedding_tools import ElmoEmbedder
elmo_embedder = ElmoEmbedder(module_url = 'https://storage.googleapis.com/az-nlp/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz')
import tensorflow as tf
import tensorflow_hub as hub
import nltk
nltk.download('punkt')
from contract_parser import ContractAnlysingContext
contractAnlysingContext = ContractAnlysingContext(elmo_embedder, renderer_)

In [0]:
!wget https://github.com/nemoware/document-parser/releases/download/1.0.5/document-parser-1.0.5-distribution.zip
!unzip document-parser-1.0.5-distribution.zip

In [0]:
import os
os.environ ['documentparser']='/content/document-parser-1.0.5'


In [0]:
from integration.word_document_parser import WordDocParser, join_paragraphs
from legal_docs import DocumentJson

wp = WordDocParser()
_out = '/content/gdrive/My Drive/GazpromOil/Всяческая документация (shared)/JSONs/contracts-19.09.26/'
contracts_filename_prefix = '/content/gdrive/My Drive/GazpromOil/Contracts/'

filenames = wp.list_filenames(contracts_filename_prefix)
print(filenames)

cnt = 0
for fn in filenames:
  cnt += 1
  print(f'reading:"{fn}"')
  short_fn = fn.split('/')[-1]

  try:
    # ------------------------
    contract = join_paragraphs(wp.read_doc(fn), fn)
    # ------------------------

    contractAnlysingContext.analyze_contract_doc(contract)
    contract = contractAnlysingContext.contract

    _path = _out + f'{short_fn}.json'
    with open(_path, 'w') as file:
      # 
      jjj = DocumentJson(contract)
      file.write(jjj.dumps())
      print(f'saved file to {_path}')
  except Exception as e:
    print(e)


### Verify random json files

In [0]:
load_and_show_json(_out+filenames[3].split('/')[-1]+'.json')

#Parse 2K sectioned contracts (Achtung, dis takes time)
- upload zipped results of document-parser first 

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
!unzip /content/jsons_phase0.zip

In [0]:
import glob
filenames = [file for file in glob.glob( "/content/jsons_phase0/**/*.json", recursive=True)] 


In [0]:

import csv, json
from bson import json_util


from contract_parser import ContractAnlysingContext
from integration.word_document_parser import join_paragraphs


def export_csv(rows, headline=['1', '2', '3', '4', '5', '6', '7', '8', '9']):
  with open(f'contracts-stats.csv', mode='w') as csv_file:
    _writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    _writer.writerow(headline)
    for l in rows:
      _writer.writerow(l)


def _parse_contract(res, doc_id, row, ctx: ContractAnlysingContext):
  contract = join_paragraphs(res, doc_id)
  ctx.analyze_contract_doc(contract)

  row[4:8] = [contract.tag_value('org.1.name'),
              contract.tag_value('org.1.alias'),
              contract.tag_value('org.2.name'),
              contract.tag_value('org.2.alias')]
  return contract

 
rows=[]
cnt=0

for fn in filenames[0:10]:
  with open(fn, 'r') as file:
    json_string = file.read()
    res = json.loads(json_string, object_hook=json_util.object_hook)

  if 'CONTRACT' == res["documentType"]:
    row = [cnt, '', None, None, None, None, None, None, fn, None]
    row[2:4] = [res["documentType"], res["documentDate"]]
    _parse_contract(res, fn,  row, contractAnlysingContext)

    rows.append(row)
    if cnt % 5 == 0: #save every 5 rows
      export_csv(rows)

    cnt+=1

In [0]:
import zipfile
import os
import sys

def zipfolder(foldername, target_dir):            
  zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
  rootlen = len(target_dir) + 1
  for base, dirs, files in os.walk(target_dir):
    for file in files:
      fn = os.path.join(base, file)
      zipobj.write(fn, fn[rootlen:])